# 方向

1. 获取试卷类型
2. 根据试卷类型提取题目和答案

## 暂时规定的类型（可逐步添加-后期也会根据试卷类型进行对齐）：
### 0. 未知的

### 1. 高考试卷/标准试卷
此类试卷是标准的考试试卷类型
例如：
```
试卷部分：
一、
1....
二、
13....
三、
16....

答案部分：
一、
1. 题干... 【答案】... 【解析】... 2....
二、
13. 题干...【答案】... 【解析】... 14....
三、
17. 题干...【答案】... 【解析】... 18....
```

### 2. 试卷解析
此类试卷是讲解试卷的类型，每道题之中存在答案
例如：
```
试卷部分：
一、
1. 题干... 【答案】... 【解析】... 2....
二、
13. 题干...【答案】... 【解析】... 14....
三、
17. 题干...【答案】... 【解析】... 18....
```


## 加载一些可能会用到的函数

In [2]:
import sys

sys.path.append('../')

from alignment_utils import one_file_per_process
from alignment_utils import extract_and_combine_numbers
from alignment_utils import extract_and_combine_numbers_in_not_start
from alignment_utils import longest_increasing_subsequence_index
from alignment_utils import find_answer_split_str
from alignment_utils import find_next_question_index
from alignment_utils import refine_answers
from alignment_utils import match_specific_from_end
from alignment_utils import answer_area_str_process
from alignment_utils import generate_answer_area_string
from alignment_utils import align_answers_in_questions
from alignment_utils import match_specific_from_start
from alignment_utils import type_of_judgment
from alignment_utils import split_question
from alignment_utils import find_continuous_sequence
from alignment_utils import extract_and_combine_numbers_in_not_start_by_number

## Load data

In [3]:
import os
import glob
from pathlib import Path
import re

examination_paper_list = []

path = Path("./docx_markdowns")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": str(file),
            "text": one_file_per_process(f.read())
        })
len(examination_paper_list)

2230

## 定义测试数据 
480 解析试卷类型

1280 标注试卷类型

444 未知

In [45]:
text = examination_paper_list[1280]["text"]
for i in text.splitlines():
    print(i)

-北师大版五年级（下）期中数学试卷（15）
一、填空题（共10小题，每小题2分，满分21分）
1．64克的![](./docx_images/media/image1.jpeg)是[]{.underline}克； 1时的![](./docx_images/media/image2.jpeg)是[]{.underline}分．
2．8.28立方分米=[]{.underline}毫升
320平方厘米=[]{.underline}平方米
[]{.underline}立方米=1580立方分米
2900毫升=[]{.underline}升．
3．一个正方体的棱长之和为48dm，它的表面积是[]{.underline}，它的体积是[]{.underline}．
4．把三个棱长4厘米的正方体拼成一个长方体，表面积减少[]{.underline}平方厘米．
5．在横线内填上适当的单位名称．
小明身高约是120[]{.underline}
一杯牛奶的容积约是250[]{.underline}
一间教室占地60[]{.underline}
一个火柴盒的体积约是8[]{.underline}．
6．找一个数的倒数（0除外），就是把它的[]{.underline}和[]{.underline}交换位置．[]{.underline}的倒数是5；0.5的倒数是[]{.underline}．
7．一个三角形的底是15cm，高是底的2倍，这个三角形的面积是[]{.underline}．
8．一件上衣200元，打八折以后的价钱是[]{.underline}元钱．
9．求运动员领奖台所占空间的大小，就是求这个领奖台的[]{.underline}．
10．如图是由同样大小的小方块堆积起来的，每个小方块的棱长是1分米，这堆小方块露在外面的面积是[]{.underline}．
![](./docx_images/media/image3.jpeg)
二、请你当小法官．（共10分）
11．将一个正方体切成两个完全相同的长方体，每个长方体的表面积是正方体表面积的一半．[]{.underline}．（判断对错）
12．7吨的![](./docx_images/media/image4.jpeg)与1吨的![](./docx_images/media/image5.jpeg)相等．[]{.underline}．
1

In [46]:
no_type_question_index = []
for i in range(len(examination_paper_list)):
    if(check_paper_type(examination_paexamination_paper_listper_list[i]["text"]) not in (1,2)):
        no_type_question_index.append(i)
    

In [33]:
len(no_type_question_index)

1632

In [36]:
no_type_question_index

[1,
 2,
 5,
 6,
 7,
 8,
 9,
 10,
 13,
 14,
 15,
 16,
 19,
 20,
 21,
 22,
 23,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 43,
 44,
 45,
 47,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 65,
 67,
 68,
 70,
 72,
 73,
 75,
 77,
 78,
 80,
 81,
 84,
 86,
 87,
 91,
 93,
 95,
 96,
 100,
 102,
 103,
 104,
 105,
 107,
 109,
 110,
 111,
 112,
 114,
 115,
 119,
 120,
 122,
 123,
 124,
 126,
 127,
 129,
 130,
 131,
 132,
 133,
 134,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 144,
 145,
 146,
 147,
 149,
 150,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 173,
 174,
 175,
 176,
 178,
 179,
 180,
 181,
 185,
 188,
 189,
 190,
 192,
 194,
 195,
 196,
 199,
 200,
 202,
 203,
 204,
 205,
 206,
 209,
 210,
 211,
 212,
 215,
 216,
 217,
 218,
 223,
 224,
 226,
 227,
 228,
 229,
 230,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 243,
 245,
 246,
 247,
 248,
 249,
 250,
 251,


In [44]:
text = examination_paper_list[0]["text"]
for i in text.splitlines():
    print(i)
    
for paper in examination_paper_list:
    

2017年全国统一高考数学试卷（理科）（新课标Ⅲ）
一、选择题：本题共12小题，每小题5分，共60分。在每小题给出的四个选项中，只有一项是符合题目要求的。
1．（5分）已知集合A={（x，y）\|x^2^+y^2^=1}，B={（x，y）\|y=x}，则A∩B中元素的个数为（）
A．3 B．2 C．1 D．0
2．（5分）设复数z满足（1+i）z=2i，则\|z\|=（）
A．![](./docx_images/media/image1.png) B．![](./docx_images/media/image2.png) C．![](./docx_images/media/image3.png) D．2
3．（5分）某城市为了解游客人数的变化规律，提高旅游服务质量，收集并整理了2014年1月至2016年12月期间月接待游客量（单位：万人）的数据，绘制了下面的折线图．
![](./docx_images/media/image4.png)
根据该折线图，下列结论错误的是（）
A．月接待游客量逐月增加
B．年接待游客量逐年增加
C．各年的月接待游客量高峰期大致在7，8月
D．各年1月至6月的月接待游客量相对于7月至12月，波动性更小，变化比较平稳
4．（5分）（x+y）（2x﹣y）^5^的展开式中的x^3^y^3^系数为 （）
A．﹣80 B．﹣40 C．40 D．80
5．（5分）已知双曲线C：![](./docx_images/media/image5.png)﹣![](./docx_images/media/image6.png)=1 （a＞0，b＞0）的一条渐近线方程为y=![](./docx_images/media/image7.png)x，且与椭圆![](./docx_images/media/image8.png)+![](./docx_images/media/image9.png)=1有公共焦点，则C的方程为（）
A．![](./docx_images/media/image10.png)﹣![](./docx_images/media/image11.png)=1 B．![](./docx_images/media/image12.png)﹣![](./docx_images/media/image13.png)=1 C．![](./docx_im

# 主体函数(检测试卷类型)

In [49]:
def find_questions_and_answer_index(lines: list[str]) -> list[int]:
    """
    获取题目和答案的lines下标
    
    判断行的开始是否为 数字+[. ．]
    """
    indexs = []
    pattern = r"^\d+[\.|\．|、]"
    for index in range(len(lines)):
        match = re.search(pattern, lines[index].replace("\\",""))
        if match: 
           indexs.append(index)
    return indexs 

## 通过条件筛选出试卷类型

In [50]:
def answer_count_total(lines, answer_keywords):
    """
    统计answer_keywords在试卷中每一行中出现的总次数
    """
    # 每道题的索引下标
    question_indexs = find_questions_and_answer_index(lines)
    answer_count = 0
    for i,question_index in enumerate(question_indexs):
        if i+1 == len(question_indexs):
            break;
        # 获取当前题的文本
        current_question = "".join(lines[question_index:question_indexs[i+1]])
        if any(keyword in current_question for keyword in answer_keywords):
            answer_count += 1
    return answer_count

In [51]:
def extract_leading_number(line):
    """
    从字符串line的开头提取数字，并匹配随后的`.`或`．`或`、`（如果存在）
    """
    match = re.search(r"^\d+[\.|\．|、]", line.replace("\\",""))
    if match:
        # 去除后面的点
        return match.group().rstrip('.').rstrip('．').rstrip('、')
    else:
        return None

In [52]:
def has_equal_subsequences(question_number_list):
    """
    传入此张试卷的所有题号，然后判断他是否对称
    对称：试卷区出现的题号 是否 答案区也出现了
    Todo
    这只是一个匹配完全标准试卷的版本，待适配
    """
    # 找到第一个元素重复的位置
    first_element = question_number_list[0]
    split_index = -1
    for i in range(1, len(question_number_list)):
        if question_number_list[i] == first_element:
            split_index = i
            break

    # 如果没有找到，返回False
    if split_index == -1:
        return False
    
    # 将数组从该位置切分为两部分
    first_subseq = question_number_list[:split_index]
    second_subseq = question_number_list[split_index:]

    # 判断这两部分是否相等
    return first_subseq == second_subseq

In [53]:
def check_standard_paper_type(lines):
    """
    检查是否符合标准试卷特征
    """
    # 寻找试卷所有的题号
    question_indexs = find_questions_and_answer_index(lines)
    question_number_list = [extract_leading_number(lines[question_index]) for question_index in question_indexs]
    
    return has_equal_subsequences(question_number_list)


In [54]:
check_standard_paper_type(text.splitlines())

True

In [55]:
def check_paper_type(text, answer_keywords=['答案']):
    """
    检查试卷类型
    
    return:
    0: 未知的
    1: 标准试卷（题目和答案分离）
    2: 试卷解析类型 （答案紧跟题目）
    -1: 试卷无答案
    """
    # 拆分成行
    lines = text.splitlines()
    #获取答案出现次数
    answer_count = answer_count_total(lines, answer_keywords)
    #获取当前试卷所有的题号
    question_indexs = find_questions_and_answer_index(lines)
    # Todo: 暂时只完成了检测 "试卷解析 "类型
    if answer_count > 0:
        if answer_count > len(question_indexs)/2:
            return 2
        elif check_standard_paper_type(lines):
            return 1
        else:
            return 0
    else:
        return -1

In [56]:
def get_paper_question_by_number(question_indexs, lines):
    question_list = []
    for i,question_index in enumerate(question_indexs):
        if i+1 == len(question_indexs):
            question_list.append("".join(lines[question_indexs[i]:]))
        else:
            question_list.append("".join(lines[question_index:question_indexs[i+1]]))
    return question_list

## 提取试题和答案 （试卷解析类型试卷type = 2） 
TODO 待补充其他类型试卷提取题目和答案

In [298]:
import datasets
from datasets import Features, Value


FEATURES = Features(
    {
#         "id": Value("string"),
        "type": Value("int32"),
        "content": Value("string")
    }
)

In [314]:
def get_analysis_of_paper(text):
    """
    获取解析试题类型试卷的试题
    """
    lines = text.splitlines()
    
    question_indexs = find_questions_and_answer_index(lines)
    
    question_list = get_paper_question_by_number(question_indesx, lines)

    return datasets.Dataset.from_dict({"type":[2]*len(question_list),"content":question_list})

In [315]:
get_analysis_of_paper(text)["content"]

['1.－3的倒数是（ ）A. 3 B. －3 C. D.【答案】D【解析】【分析】根据倒数的定义求解．【详解】-3的倒数为．\\故选：D．【点睛】本题考查了倒数，分子分母交换位置是求倒数的关键．',
 '2.下列计算正确的是（ ）A. B. C. D.【答案】D【解析】【分析】分别利用合并同类项法则以同底数幂的乘除法运算法则计算得出答案．【详解】解：A、，不能合并，故此选项错误；\\B、，无法计算，故此选项错误；\\C、，故此选项错误；\\D、，故此选项正确；\\故选：D．【点睛】本题考查同底数幂的乘除法运算以及合并同类项，正确掌握运算法则是解题关键．',
 '3.教育部近日发布了2019年全国教育经费执行情况统计快报，经初步统计，2019年全国教育经费总投入为50175亿元，比上年增长8.74%，将50175亿用科学记数法表示为（ ）A. B. C. D.【答案】A【解析】【分析】科学记数法的表示形式为的形式，其中，n为整数，确定n的值时，要看把原数变成a时，小数点移动了多少位，n的绝对值与小数点移动的位数相同；当原数的绝对值\\1时，n是正数；当原数的绝对值\\<1时，n是负数．【详解】解：将数字50175亿用科学记数法表示为故本题选A．【点睛】本题主要考查了科学记数法的表示方法，科学记数法的表示形式为的形式，其中，n为整数，表示时关键要正确确定a与n的值．',
 '4.如图，，则下列结论错误的是（ ）![](./docx_images/media/image20.png)A. B. C. D.【答案】C【解析】【分析】由可对A进行判断；根据三角形外角的性质可对B进行判断；求出∠C，根据大角对大边，小角对小边可对D进行判断；求出可对C进行判断．【详解】，，故选项A正确；，，又，，故选项B正确；，，，，故选项D正确；，，而，故选项C错误．故选C．【点睛】此题主要考查了平行线的判定与性质，三角形外角的性质等知识，熟练掌握性质与判定是解答此题的关键．',
 '5.如图所示，正方体的展开图为（ ）![](./docx_images/media/image42.png)A. ![](./docx_images/media/image43.png) B. ![](./docx_images/media/image44.png)C![](./docx_images/medi

## 提取试题和答案 （标准类型试卷type = 1）

In [64]:
def get_answer_area(text):
    """
    获取标准试卷答案区
    返回每题答案的集合
    """
    
    lines = text.splitlines()
    for i in range(len(lines)):
        if bool(re.search(r'参考答案|试卷解析', lines[i])):
            question_number = find_questions_and_answer_index(lines[:i])
            question_list = get_paper_question_by_number(question_number, lines)
            
            return question_list
    return None

In [65]:
for i in get_answer_area(text):
    print(i)
    print()

1．64克的![](./docx_images/media/image1.jpeg)是[]{.underline}克； 1时的![](./docx_images/media/image2.jpeg)是[]{.underline}分．

2．8.28立方分米=[]{.underline}毫升320平方厘米=[]{.underline}平方米[]{.underline}立方米=1580立方分米2900毫升=[]{.underline}升．

3．一个正方体的棱长之和为48dm，它的表面积是[]{.underline}，它的体积是[]{.underline}．

4．把三个棱长4厘米的正方体拼成一个长方体，表面积减少[]{.underline}平方厘米．

5．在横线内填上适当的单位名称．小明身高约是120[]{.underline}一杯牛奶的容积约是250[]{.underline}一间教室占地60[]{.underline}一个火柴盒的体积约是8[]{.underline}．

6．找一个数的倒数（0除外），就是把它的[]{.underline}和[]{.underline}交换位置．[]{.underline}的倒数是5；0.5的倒数是[]{.underline}．

7．一个三角形的底是15cm，高是底的2倍，这个三角形的面积是[]{.underline}．

8．一件上衣200元，打八折以后的价钱是[]{.underline}元钱．

9．求运动员领奖台所占空间的大小，就是求这个领奖台的[]{.underline}．

10．如图是由同样大小的小方块堆积起来的，每个小方块的棱长是1分米，这堆小方块露在外面的面积是[]{.underline}．![](./docx_images/media/image3.jpeg)二、请你当小法官．（共10分）

11．将一个正方体切成两个完全相同的长方体，每个长方体的表面积是正方体表面积的一半．[]{.underline}．（判断对错）

12．7吨的![](./docx_images/media/image4.jpeg)与1吨的![](./docx_images/media/image5.jpeg)相等．[]{.underline}．

13．两个分数相乘，积一定小于其中任何一个分数．[]{.underline}．（判断对错）

In [42]:
def get_question_or_answer(lines):
    question_number = find_questions_and_answer_index(lines)
    print(question_number)
    question_list = get_paper_question_by_number(question_number, lines)
    return question_list

In [43]:
print(get_question_or_answer(get_answer_area(text)))

[2, 9, 23, 35, 41, 56, 61, 69, 77, 82, 93, 100, 109, 115, 121, 128, 134, 141, 149, 157, 165, 177, 197, 214, 226, 235, 242, 248, 259]
['1．64克的![](./docx_images/media/image39.jpeg)是[]{.underline}![](./docx_images/media/image40.jpeg)[]{.underline}克； 1时的![](./docx_images/media/image41.jpeg)是[28]{.underline}分．【考点】分数乘法．【分析】求64克的![](./docx_images/media/image39.jpeg)是多少克，用64乘![](./docx_images/media/image39.jpeg)；求1时的![](./docx_images/media/image41.jpeg)是多少分，用60分钟乘![](./docx_images/media/image41.jpeg)，由此解答即可．【解答】解：64×![](./docx_images/media/image39.jpeg)=![](./docx_images/media/image40.jpeg)（克）；1时=60分，60×![](./docx_images/media/image41.jpeg)=28（分）．故答案为：![](./docx_images/media/image40.jpeg)，28．', '2．8.28立方分米=[8280]{.underline}毫升320平方厘米=[0.032]{.underline}平方米[1.58]{.underline}立方米=1580立方分米2900毫升=[2.9]{.underline}升．【考点】体积、容积进率及单位换算；面积单位间的进率及单位换算．【分析】（1）高级单位立方分米化低级单位毫升乘进率1000．（2）低级单位平方厘米化高级单位平方米除以进率10000．（3）低级单位立方分米化高级单位立方米除以进率1000．（4）低级单位毫升化高级单位升除以进率1000．【解答】解：（1）8.28立方分米=8280毫升；（2）320平方厘米=0.032平方米

In [41]:
for question_or_answer in get_question_and_answer(text):
    print(question_or_answer)
    print("==========================")

NameError: name 'get_question_and_answer' is not defined

## 对齐函数
根据不同类型试卷进行不同的对齐逻辑
```
标准试卷:
按序号来对齐每道题

解析型试卷：
按【答案】等字样进行分割
```

In [280]:
def alignment_type_two(question_list):
    question_and_answer = []
    for question in question_list:
        part = re.split(r'(?=\【答案】|\【考点】|\【分析】)', question, 1)
        question_and_answer.append({"question":part[0], "answer": part[1]})
    return question_and_answer

In [285]:
for alignment in alignment_type_two(get_question_and_answer(text)):
    print(alignment["question"])
    print()
    print(alignment["answer"])
    print("===============")

1.－3的倒数是（ ）A. 3 B. －3 C. D.

【答案】D【解析】【分析】根据倒数的定义求解．【详解】-3的倒数为．\故选：D．【点睛】本题考查了倒数，分子分母交换位置是求倒数的关键．
2.下列计算正确的是（ ）A. B. C. D.

【答案】D【解析】【分析】分别利用合并同类项法则以同底数幂的乘除法运算法则计算得出答案．【详解】解：A、，不能合并，故此选项错误；\B、，无法计算，故此选项错误；\C、，故此选项错误；\D、，故此选项正确；\故选：D．【点睛】本题考查同底数幂的乘除法运算以及合并同类项，正确掌握运算法则是解题关键．
3.教育部近日发布了2019年全国教育经费执行情况统计快报，经初步统计，2019年全国教育经费总投入为50175亿元，比上年增长8.74%，将50175亿用科学记数法表示为（ ）A. B. C. D.

【答案】A【解析】【分析】科学记数法的表示形式为的形式，其中，n为整数，确定n的值时，要看把原数变成a时，小数点移动了多少位，n的绝对值与小数点移动的位数相同；当原数的绝对值\1时，n是正数；当原数的绝对值\<1时，n是负数．【详解】解：将数字50175亿用科学记数法表示为故本题选A．【点睛】本题主要考查了科学记数法的表示方法，科学记数法的表示形式为的形式，其中，n为整数，表示时关键要正确确定a与n的值．
4.如图，，则下列结论错误的是（ ）![](./docx_images/media/image20.png)A. B. C. D.

【答案】C【解析】【分析】由可对A进行判断；根据三角形外角的性质可对B进行判断；求出∠C，根据大角对大边，小角对小边可对D进行判断；求出可对C进行判断．【详解】，，故选项A正确；，，又，，故选项B正确；，，，，故选项D正确；，，而，故选项C错误．故选C．【点睛】此题主要考查了平行线的判定与性质，三角形外角的性质等知识，熟练掌握性质与判定是解答此题的关键．
5.如图所示，正方体的展开图为（ ）![](./docx_images/media/image42.png)A. ![](./docx_images/media/image43.png) B. ![](./docx_images/media/image44.png)C![](./docx_images/media/image45.wmf) ![

## 整体流程
1. 判断试卷类型
2. 根据试卷类型对题目和答案进行提取
3. 根据试卷类型提取出来的题目和答案对齐

In [251]:
# 判断试卷类型
paper_tpye = check_paper_type(text)
# 假设试卷类型为试卷解析类型（2）
question_list = get_question_and_answer(text)
# 提取
alignment_type_two(question_list)

[{'question': '1.－3的倒数是（ ）A. 3 B. －3 C. D.',
  'answer': '【答案】D【解析】【分析】根据倒数的定义求解．【详解】-3的倒数为．\\故选：D．【点睛】本题考查了倒数，分子分母交换位置是求倒数的关键．'},
 {'question': '2.下列计算正确的是（ ）A. B. C. D.',
  'answer': '【答案】D【解析】【分析】分别利用合并同类项法则以同底数幂的乘除法运算法则计算得出答案．【详解】解：A、，不能合并，故此选项错误；\\B、，无法计算，故此选项错误；\\C、，故此选项错误；\\D、，故此选项正确；\\故选：D．【点睛】本题考查同底数幂的乘除法运算以及合并同类项，正确掌握运算法则是解题关键．'},
 {'question': '3.教育部近日发布了2019年全国教育经费执行情况统计快报，经初步统计，2019年全国教育经费总投入为50175亿元，比上年增长8.74%，将50175亿用科学记数法表示为（ ）A. B. C. D.',
  'answer': '【答案】A【解析】【分析】科学记数法的表示形式为的形式，其中，n为整数，确定n的值时，要看把原数变成a时，小数点移动了多少位，n的绝对值与小数点移动的位数相同；当原数的绝对值\\1时，n是正数；当原数的绝对值\\<1时，n是负数．【详解】解：将数字50175亿用科学记数法表示为故本题选A．【点睛】本题主要考查了科学记数法的表示方法，科学记数法的表示形式为的形式，其中，n为整数，表示时关键要正确确定a与n的值．'},
 {'question': '4.如图，，则下列结论错误的是（ ）![](./docx_images/media/image20.png)A. B. C. D.',
  'answer': '【答案】C【解析】【分析】由可对A进行判断；根据三角形外角的性质可对B进行判断；求出∠C，根据大角对大边，小角对小边可对D进行判断；求出可对C进行判断．【详解】，，故选项A正确；，，又，，故选项B正确；，，，，故选项D正确；，，而，故选项C错误．故选C．【点睛】此题主要考查了平行线的判定与性质，三角形外角的性质等知识，熟练掌握性质与判定是解答此题的关键．'},
 {'question': '5.如图所示，正方体的展开图为（ ）![](.

In [ ]:
def run(text):
    paper_type = check_paper_type(text)
    
    if paper_type == 1:
    
    elif paper_type == 2:

## 暂时用不到的代码

In [232]:
def find_one_question_index(lines):
    indexs = []
    pattern = r'^(一|二|三|四|五|六|七|八|九|十)'
    for index in range(len(lines)):
        match = re.search(pattern, lines[index])
        if match: 
           indexs.append(index)
            
    return indexs